In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import math
from PIL import Image
%matplotlib inline

from tqdm import tqdm_notebook as tqdm

# Preprocessing

In [ ]:
train=pd.read_csv('train.csv')
train.head()

In [ ]:
train.dtypes

In [ ]:
train.shape

In [ ]:
test=pd.read_csv('test.csv')
test.head()

In [ ]:
test.dtypes

In [ ]:
test.shape

In [ ]:
train['month'].unique()

In [ ]:
sorted(test['month'].unique())

In [ ]:
train[train['month']==12].sort_values('cust')

In [ ]:
train[train['cust']=='M003213142']

In [ ]:
train[train['cust']=='M011706752']

# FE
- |해당월 소비 금액 - 월별 평균 소비 금액
-  |해당 제휴사 소비 금액 - 제휴사별 평균 소비 금액
- | 해당 월, 제휴사 소비 금액 - 제휴사별 , 월별 평균 소비 금액

</br>


- 거주 지역 코드와 구매 지역 다른지
- 주말, 주중
- 직전 달 소비 금액
- 고객 예측 소득
- 고객 가치

## Month_diff

In [ ]:
mon=train.groupby('month')['buying'].mean().reset_index()
mon.rename(columns={'buying':'buy_mean'},inplace=True)
mon

In [ ]:
# train
mrg=pd.merge(train,mon,on='month',how='left')
mrg['mon_buy_diff']=abs(mrg['buying']-mrg['buy_mean_'])
mrg

In [ ]:
train['month_diff']=mrg['mon_buy_diff']
train.head()

In [ ]:
# test : 요약 통계량 값은 train데이터에서만 사용합니다
# 왜냐하면 test데이터는 unknown값이기 때문! -> data leakage
mrg=pd.merge(test,mon,on='de_month',how='left')
mrg['mon_buy_diff']=abs(mrg['buy_am']-mrg['buy_mean_'])
mrg

In [ ]:
test['month_diff']=mrg['mon_buy_diff']
test.head()

## cop_diff

In [ ]:
co=train.groupby('cop')['buying'].mean().reset_index()
co.rename(columns={'buying':'buy_mean_'},inplace=True)
co

In [ ]:
# train
mrg2=pd.merge(train,co,on='cop',how='left')
mrg2['co_buy_diff']=abs(mrg2['buy_am']-mrg2['buy_mean_'])
mrg2

In [ ]:
train['cop_diff']=mrg2['co_buy_diff']
train.head()

In [ ]:
# test
mrg2=pd.merge(test,co,on='cop',how='left')
mrg2['co_buy_diff']=abs(mrg2['buy_am']-mrg2['buy_mean_'])
mrg2

In [ ]:
test['cop_diff']=mrg2['co_buy_diff']
test.head()

## c_m_diff

In [ ]:
cm=train.groupby(['cop','month'])['buying'].mean().reset_index()
cm.rename(columns={'buying':'buy_mean_'},inplace=True)
cm.head()

In [ ]:
# train
mrg3=pd.merge(train,cm,on=['cop','month'],how='left')
mrg3['cm_buy_diff']=mrg3['buying']-mrg3['buy_mean_']
mrg3

In [ ]:
train['cm_diff']=mrg3['cm_buy_diff']
train.head()

In [ ]:
# test
mrg3=pd.merge(test,cm,on=['cop','month'],how='left')
mrg3['cm_buy_diff']=mrg3['buying']-mrg3['buy_mean_']
mrg3

In [ ]:
test['cm_diff']=mrg3['cm_buy_diff']
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

## Zon diff

In [ ]:
# 거주지와 소비지 다른지
train['loc_diff']=[1 if val!=train['loc_hlv'][num] else 0 for num,val in enumerate(train['loc_resid'])]
test['loc_diff']=[1 if val!=test['loc_hlv'][num] else 0 for num,val in enumerate(test['loc_resid'])]

## weekend

In [ ]:
# train
dt=pd.to_datetime(train['fin_dt'],format="%Y%m%d")
wk_num=dt.apply(lambda x:date.weekday(x))
wk_num

In [ ]:
weekend=[0 if i<5 else 1 for i in wk_num]
train['weekend']=weekend

In [ ]:
# test
dt=pd.to_datetime(test['fin_dt'],format="%Y%m%d")
wk_num=dt.apply(lambda x:date.weekday(x))
wk_num

In [ ]:
weekend=[0 if i<5 else 1 for i in wk_num]
test['weekend']=weekend

In [ ]:
train.shape

In [ ]:
test.shape

## Lag_buy 
- 직전달 소비 금액
- 만약, 하나의 고객이 모든 달의 결제 데이터가 없을 경우, 직전달은 어떤 달을 넣어야하나?
  - 1) 결측치
  - 2) 가장 최근인 월로 대체 -> 이 경우 채택


In [ ]:
tot=pd.concat([train,test]).reset_index()
tot

In [ ]:
buy_month=tot.sort_values(['month','cust'],ascending=True).groupby(['cust','month'])['buying'].sum().reset_index()
sh=buy_month[['cust','buying']]

In [ ]:
# train
# 고객별 이전 달 소비 금액
buy_month=tot.sort_values(['month','cust'],ascending=True).groupby(['cust','month'])['buying'].sum().reset_index()
sh=buy_month[['cust','buying']]
buy_month['buy_mnt']=sh['buying']
bm=buy_month[['cust','month','buy_mnt']]
train1=pd.merge(train,bm,on=['cust','month'],how='left')
train1

In [ ]:
# test
# 고객별 이전 달 소비 금액
test1=pd.merge(test,bm,on=['cust','month'],how='left')
test1.head()

In [ ]:
train1.isna().sum()

In [ ]:
test1.isna().sum()

In [ ]:
train1.shape

In [ ]:
test1.shape

## Buy level & Lag_income
- 고객 예측 소득
  - 직전 달 소비액을 통한 이전 달 소득 추정값 구하기
  - https://kostat.go.kr/portal/korea/kor_nw/1/4/3/index.board?bmode=read&bSeq=&aSeq=389709&pageNo=1&rowNum=10&navCount=10&currPg=&searchInfo=&sTarget=title&sTxt=
  - https://www.index.go.kr/unify/idx-info.do?idxCd=4229
  - 아래의 표를 이용해 고객의 월별 소비액을 구하고, 소득 분위에 따른 월별 소비액을 구해 그 값의 차의 절댓값이 가장 적은 분위수에 할당하기

- 소비액

In [ ]:
############ 소비액 ############
# 2021년 기준 평균 가구원 수 2.3명 -> 2.3으로 나누기
# 1분기 소비액
# [1분위, 2분위, 3분위, 4분위, 5분위]
quart1=[1125000,1623000,3618000,2845000,4282000]
quart1=[round(i/2.3,4) for i in quart1]
#quart1_mean=2419000

# 2분기 개인별 소비액
quart2=[1153000, 1674000, 2194000, 2915000, 4435000]
qurt2=[round(i/2.3,4) for i in quart2]
#quart2_mean=2475000

# 3분기 개인별 소비액
quart3=[1178000, 1736000, 2355000, 3088000, 4361000]
quart3=[round(i/2.3,4) for i in quart3]
#quart3_mean=2544000

# 4분기 개인별 소비액
quart4=[1195000,1759000,2383000,3131000,4266000]
quart4=[round(i/2.3,4) for i in quart4]
#quart4_mean=2547000

In [ ]:
# 분기 -> 한달

# 1,2,3
q1=[round(i/3,4) for i in quart1]

# 4,5,6
q2=[round(i/3,4) for i in quart2]

# 7,8,9
q3=[round(i/3,4) for i in quart3]

# 10,11,12
q4=[round(i/3,4) for i in quart4]

In [ ]:
print(q1)
print(q2)
print(q3)
print(q4)

In [ ]:
lst=q1*3
lst.extend(q2*3)
lst.extend(q3*3)
lst.extend(q4*3)
lst[:10]

In [ ]:
len(lst)

In [ ]:
buy_st=pd.DataFrame({'month':[i for i in range(1,13) for _ in range(5)], # 달
              'buy_level':[i for _ in range(12) for i in range(1,6)], # 분위수
              'buy_mean':lst})
buy_st[:5]

In [ ]:
buy_month_=buy_month.sort_values('de_month').reset_index(drop=True)
buy_month_

In [ ]:
buy_st[:5]

In [ ]:
buy_month_

In [ ]:
level=[]
for mon in range(1,13):
  t=list(buy_month_[buy_month_['de_month']==mon]['buy_am'])
  st=list(buy_st[buy_st['month']==mon]['buy_mean'])

  for i in t:
    l=[]
    for j in st:
      l.append(abs(i-j))
    level.append(l.index(min(l))+1)

In [ ]:
[level.count(i) for i in range(1,6)]

In [ ]:
#[level.count(i) for i in range(1,6)]

In [ ]:
plt.hist(level)

In [ ]:
buy_month_['buy_level']=level

In [ ]:
buy_month_

- 소득액

- 왜 /2.3 안하나요? 
  - 모든 가구 구성원이 수입을 버는 것이 아니기 때문에 가구 소득을 개인 소득으로 생각합니다.
  - 앞서 소비액은 모든 가구 구성원이 소비를 할 수 있기 때문에 /2.3 하였음.

In [ ]:
############ Income ############
# 1분기 소득
quart1=[910000,2301000,3618000,5370000,9714000]
#quart1=[round(i/2.3,4) for i in quart1]

# 2분기
quart2=[966000,2365000,3661000,5192000,9241000]
#qurt2=[round(i/2.3,4) for i in quart2]

# 3분기
quart3=[1142000,2647000,4018000,5792000,10037000]
#quart3=[round(i/2.3,4) for i in quart3]

# 4분기
quart4=[1058000,2531000,3879000,5612000,10130000]
#quart4=[round(i/2.3,4) for i in quart4]


In [ ]:
lst=quart1*3
#lst
lst.extend(quart2*3)
lst.extend(quart3*3)
lst.extend(quart4*3)
lst[:10]

In [ ]:
incom_st=pd.DataFrame({'de_month':[i for i in range(1,13) for _ in range(5)], 
                       'buy_level':[i for _ in range(12) for i in range(1,6)], 
                       'income_mean':lst})
incom_st[:5]

In [ ]:
incom_st[::][:5]

In [ ]:
buy_month_=pd.merge(buy_month_,incom_st,on=['de_month','buy_level'],how='left')
buy_month_

In [ ]:
buy_month1=buy_month_.sort_values(['cust','de_month']).reset_index(drop=True)
buy_month1[:5]

In [ ]:
sh1=buy_month1[['cust','income_mean']]#.groupby(['cust']).shift(1)
sh1
#sh=buy_month[['cust','buy_am']].groupby('cust').shift(1)

In [ ]:
buy_month1['lag_income']=sh1['income_mean']
buy_month1

In [ ]:
buy_month1[::][:12]

In [ ]:
buy_month1.rename(columns={'lag_income':'income_mnt'},inplace=True)

In [ ]:
buy_month1[:5]

In [ ]:
# 필요한 컬럼 추출
buy_month2=buy_month1[['cust','de_month','buy_level','buy_mnt','income_mnt']]
buy_month2

In [ ]:
train2=pd.merge(train,buy_month2,on=['cust','de_month'],how='left')
test2=pd.merge(test,buy_month2,on=['cust','de_month'],how='left')

In [ ]:
train2.shape

In [ ]:
test2.shape

# CLTV (종속변수)
- 종속변수!
- https://www.openads.co.kr/content/contentDetail?contsId=5505
- 한달 단위
- 이탈률 및 매출은 제휴사 별로 다르게 계산

In [ ]:
tot['cop'].unique()
# A: 유통사
# B: 숙박업종
# C: 엔터테인먼트
# D: F&B
# E: 렌탈업종

In [ ]:
# 방문 횟수 추가
# 이는 나중에 groupby해서 sum 할 때 하나의 고객이 제휴사를 몇 번 방문했는 지 알 수 있음
tot['vst']=1 
tot.head()

In [ ]:
### 평균 구매 가치 (Average purchase value) ###
# 총매출 / 구매횟수
cop_month=tot.groupby(['cop','month']).agg({'cust':lambda x:len(x.unique()),
                                                'buy_ct':'sum','buying':'sum','vst':'sum'}).reset_index()
cop_month['apv']=cop_month['buying']/cop_month['vst']
cop_month[:10]

In [ ]:
### 평균 구매 빈도율 (Average purchase frequency rate) ###
# 총 구매 횟수 / 구매 고객수
cop_month['apfr']=cop_month['vst']/cop_month['cust']
cop_month[:10]

In [ ]:
### 고객 가치 (Customer value) ###
# 고객 별 평균 구매 액수[mean(buy_am)] * 평균 구매 빈도율[apfr]
# 모든 고객에 대해 계산하고, 평균치 내기
val_df=tot.groupby(['cust','cop','month']).agg({'buy_ct':'sum','buying':'mean','vst':'sum'}).reset_index()
val_df
val=pd.merge(val_df,cop_month[['cop','month','apfr']],on=['cop','month'],how='left')
val['cv']=val['buying']*val['apfr']
val

In [ ]:
### 평균 고객 수명 (Average customer lifespan ) ###
# 1/ 고객 이탈률
# https://www.quora.com/Why-is-1-churn-rate-average-customer-lifetime

c_df=tot.groupby(['cop','month']).agg({'cust':lambda x:x.unique()}).reset_index()
c_sh=c_df.copy()
c_df['lag_cust']=c_sh[['cop','cust']].groupby('cop').shift(1)

lifespan=[]
cnt=0
for c,lg in zip(c_df['cust'],c_df['lag_cust']):
  if cnt%12==0:
    lifespan.append(np.nan)
  else:
    churn=1-len(set(lg).intersection(set(c)))/len(set(lg))
    lifespan.append(1/churn)
  cnt+=1

c_df['lifespan']=lifespan
c_df[:5]

In [ ]:
### 고객 생애가치 (CLTV) ###
# customer value * average customer lifespan
fin_df=pd.merge(val,c_df[['cop','month','lifespan']],on=['cop','month'],how='left')
fin_df['cltv']=fin_df['cv']*fin_df['lifespan']
fin_df

In [ ]:
# train
final=pd.merge(tot,fin_df[['cust','cop','month','cltv']],on=['cust','cop','month'],how='left')
final[:5]

In [ ]:
del final['vst']

In [ ]:
final.shape

In [ ]:
final.isna().sum()

In [ ]:
list(final[:3826823]['cust'])==list(train2['cust'])

In [ ]:
list(final[:3826823]['month'])==list(train2['month'])

In [ ]:
list(final[:3826823]['cop'])==list(train2['cop'])

In [ ]:
list(final[3826823:]['cust'])==list(test2['cust'])

In [ ]:
list(final[3826823:]['month'])==list(test2['month'])

In [ ]:
list(final[3826823:]['cop'])==list(test2['cop'])

In [ ]:
train2.isna().sum()

In [ ]:
train2.shape

In [ ]:
train2['cltv']=list(final[:3826823]['cltv'])
train2.shape

In [ ]:
train2.isna().sum()

In [ ]:
print(final[:3826823]['cltv'].isna().sum())
print(final[3826823:]['cltv'].isna().sum())

In [ ]:
test2['cltv']=list(final[3826823:]['cltv'])
test2.shape

In [ ]:
test2.isna().sum()

In [ ]:
#train2.to_csv('train_fe.csv',index=False)
#test2.to_csv('test_fe.csv',index=False)

In [ ]:
#tr=pd.read_csv('train_fe.csv')
tr=train2.copy()
print(tr.shape)
print(tr.isna().sum())

In [ ]:
#ts=pd.read_csv('test_fe.csv')
ts=test2.copy()
print(ts.shape)
print(ts.isna().sum())

# NA value
- 제거

In [ ]:
train_df=tr[~tr['buy_mnt'].isna()].reset_index(drop=True)
train_df.shape

In [ ]:
train=train_df.copy()

# Remove col
- 제거해야할 컬럼: fin_dt, de_year

In [ ]:
train_df=train[['cust', 'chnl_dv', 'de_hr', 'buy_am', 'buy_ct', 'time_diff',
        'de_month', 'de_day', 'ma_fem_dv', 'ages', 'zon_hlv_resid',
       'zon_hlv', 'cop', 'month_diff', 'cop_diff', 'cm_diff', 'zon_hlv_diff',
       'weekend', 'buy_level', 'buy_mnt', 'income_mnt', 'cltv']]

test_df=ts[['cust', 'chnl_dv', 'de_hr', 'buy_am', 'buy_ct', 'time_diff',
        'de_month', 'de_day', 'ma_fem_dv', 'ages', 'zon_hlv_resid',
       'zon_hlv', 'cop', 'month_diff', 'cop_diff', 'cm_diff', 'zon_hlv_diff',
       'weekend', 'buy_level', 'buy_mnt', 'income_mnt', 'cltv']]

## categorical var
- cust : 1번 ~ n번까지 매핑 (비복원 추출)
- ma_fem_dv : 여성, 남성 -> 원핫인코딩
- ages : '대' 제거후 int 형으로
- zon_hlv_resid : one-hot
- zon_hlv : one-hot
- cop: one-hot 
- chnl_dv: one-hot

In [ ]:
# ma_fem_dv

# train
train_df.rename(columns={'ma_fem_dv':'gender'},inplace=True)
train_df['gender']=train_df['gender'].map({'여성':'female','남성':'male'})

# test
test_df.rename(columns={'ma_fem_dv':'gender'},inplace=True)
test_df['gender']=test_df['gender'].map({'여성':'female','남성':'male'})


In [ ]:
# ages
train_df['ages']=train_df['ages'].apply(lambda x:str(x)[:-1])
test_df['ages']=test_df['ages'].apply(lambda x:str(x)[:-1])

In [ ]:
train_df['ages']=train_df['ages'].astype(int)
test_df['ages']=test_df['ages'].astype(int)

In [ ]:
# channel
ch_dic={1:'offline',2:'online'}
train_df['channel']=train_df['channel'].map(ch_dic)
test_df['channel']=test_df['channel'].map(ch_dic)

In [ ]:
# one-hot encoding 
df_tr=pd.get_dummies(train_df, columns=['gender','cop','channel'])
df_ts=pd.get_dummies(test_df, columns=['gender','cop','channel'])

In [ ]:
# cust
n=len(df_tr['cust'].unique())
np.random.seed(2022) # Randomness 추가함 -> random state 다르게 하며 모델 비교!!!
rn_lst=np.random.choice(n,n,replace=False)

cust_dic={v:0 for v in df_tr['cust'].unique()}
for i,c in enumerate(cust_dic):
  cust_dic[c]=rn_lst[i]

df_tr['cust']=df_tr['cust'].map(cust_dic)
df_ts['cust']=df_ts['cust'].map(cust_dic)

In [ ]:
# zon_hlv
df_tr2=pd.get_dummies(df_tr,columns=['loc','loc_resid'])
df_ts2=pd.get_dummies(df_ts,columns=['loc','loc_resid'])

In [ ]:
df_tr2.to_csv('train_fin_time.csv',index=False)
df_ts2.to_csv('test_fin_time.csv',index=False)